<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('/datasets/Churn.csv', index_col=0)
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           9091 non-null   float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1.1+ MB


In [4]:
df['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

В столбце "Tenure" наблюдаем пропуски, заполним их медианой, чтобы несильно исказить данные.

In [5]:
df['Tenure'] = df['Tenure'].fillna(df['Tenure'].median())
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 1 to 10000
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  float64
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 1.1+ MB


В данные для обучения не будем брать столбцы "CustomerId" и "Surname", т.к. связи с целевым признаком они не имеют и поведут к ошибкам в обучении

In [6]:
#Подготовка категориальных признаков

df_ohe = df.drop(['CustomerId', 'Surname'], axis=1)
df_ohe = pd.get_dummies(df_ohe, drop_first=1)
df_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
RowNumber,,,,,,,,,,,,
1,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
2,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
3,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
4,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
5,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9996,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9997,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9998,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0


In [7]:
#Разделение данных на обучающие, валидационную и тестовую выборки
target = df_ohe['Exited']
features = df_ohe.drop('Exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(features_valid, target_valid, 
                                                                              test_size=0.5, random_state=12345)

In [8]:
# Подготовка количественных признаков

numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [9]:
features_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
RowNumber,,,,,,,,,,,
7480,-0.886751,-0.373192,1.082277,1.232271,-0.891560,1,0,-0.187705,0,1,1
3412,0.608663,-0.183385,1.082277,0.600563,-0.891560,0,0,-0.333945,0,0,0
6028,2.052152,0.480939,-0.737696,1.027098,0.830152,0,1,1.503095,1,0,1
1248,-1.457915,-1.417129,0.354288,-1.233163,0.830152,1,0,-1.071061,0,0,1
3717,0.130961,-1.132419,-1.101690,1.140475,-0.891560,0,0,1.524268,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4479,-1.073677,-0.752805,-0.373701,-1.233163,0.830152,0,1,-1.278361,0,0,1
4095,-1.447531,-0.942612,1.810266,-1.233163,0.830152,0,1,-1.281307,0,0,1
3493,0.027113,0.575842,-0.009707,-0.310229,-0.891560,0,1,-0.903158,0,0,0


## Исследование задачи

Попробуем найти необходимую модель без исследования баланса классов

In [ ]:
#Дерево решений

best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
for depth in range(1, 16):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_valid)
    result = f1_score(target_valid, predictions)
    if result > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = result
        best_depth = depth
        
print('F1 лучшей модели дерева решений:', best_result_tree, 'с глубиной дерева равной', best_depth)

probabilities_valid = best_model_tree.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)

In [ ]:
#Случайный лес

best_result_forest = 0
best_depth_forest = 0
best_est_forest = 0
best_model_forest = None
for est in range(10, 101, 10):
    for depth in range(1, 13):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        scores = cross_validate(model_forest, features, target, cv=5)['test_score']
        result = scores.sum()/len(list(scores))
        if result > best_result_forest:
            best_result_forest = result
            best_depth_forest = depth
            best_est_forest = est
            best_model_forest = model_forest

print('Лучшая средняя оценка по результатам кросс-валидации равна', best_result_forest, 
      'у случайного леса с количеством деревьев равным', best_est_forest, 'и глубиной равной', best_depth_forest)

best_model_forest.fit(features_train, target_train)
predictions = best_model_forest.predict(features_valid)
result_f1 = f1_score(target_valid, predictions)
print('F1 лучшей модели cлучайного леса:', result_f1)

probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)

In [ ]:
model_linear = LogisticRegression(solver='liblinear', random_state=12345)
model_linear.fit(features_train, target_train)
predicted_valid = model_linear.predict(features_valid)
result = f1_score(target_valid, predicted_valid)

print('F1 модели логистической регрессии на валидационной выборке:', result)

probabilities_valid = model_linear.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)

Ни один вид модели не дал нужное значение F1-меры, зато мы выяснили, что модель случайного леса выдаёт наибольшее значение, значит с ней и будем работать в дальнейшем, т.к. по условию задачи сказано "Постройте модель с предельно большим значением F1-меры.".  

Исследуем баланс классов

In [ ]:
predictions = best_model_forest.predict(features_valid)
print(confusion_matrix(target_valid, predictions))

In [ ]:
print('Количество элементов отрицательного класса')
print(target_train.loc[target_train == 0].count())

print('Количество элементов положительного класса')
print(target_train.loc[target_train == 1].count())

ratio = round(target_train.loc[target_train == 0].count()/target_train.loc[target_train == 1].count())

print('Сотношение количества элементов отрицательного класса к количеству элементов положительного примерно равно:', ratio)

<b>Вывод</b>

- Модель случайного леса наиболее подходит к решению данной задачи.
- В данных наблюдается сильный дисбаланс классов. Соотношение количества элементов отрицательного класса к количеству элементов положительного примерно равно 4:1, что сказывается на предсказаниях положительного класса.

## Борьба с дисбалансом

- Попробуем уравновесить классы аргументом class_weight='balanced'
- Попробуем увеличить выборку положительного класса умножением её на коэф. соотношения, чтобы привести соотношение приблизительно к 1:1
- Попробуем уменьшить выборку отрицательного класса делением её на коэф. соотношения, чтобы привести соотношение приблизительно к 1:1

In [ ]:
#Уравновешивание классов

model = RandomForestClassifier(random_state=12345, n_estimators= best_est_forest, 
                               max_depth=best_depth_forest, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print('Метрика F1-меры после взвешивания классов равна', f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)
print(confusion_matrix(target_valid, predicted_valid))

In [ ]:
#Функция увеличения выборки

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
#Проверка работы функции

print(features_train.shape)
print(target_train.shape)

features_upsampled, target_upsampled = upsample(features_train, target_train, ratio)

print(features_upsampled.shape)
print(target_upsampled.shape)

In [ ]:
#Увеличение выборки и проверка на модели случайного леса

scaler = StandardScaler()
scaler.fit(features_upsampled[numeric])
features_upsampled[numeric] = scaler.transform(features_upsampled[numeric])

model = RandomForestClassifier(random_state=12345, n_estimators=best_est_forest, max_depth=best_depth_forest)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print('Метрика F1-меры после увеличения выборки положительного класса равна', f1_score(target_valid, predicted_valid))

probabilities_valid = model_linear.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)
print(confusion_matrix(target_valid, predicted_valid))

In [ ]:
#Функция уменьшения выборки отрицательного класса

def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
#Проверка работы функции

features_downsampled, target_downsampled = downsample(features_train, target_train, 1/ratio)

print(features_downsampled.shape)
print(target_downsampled.shape)

In [ ]:
#Уменьшение выборки отрицательного класса и проверка на модели случайного леса

scaler = StandardScaler()
scaler.fit(features_downsampled[numeric])
features_downsampled[numeric] = scaler.transform(features_downsampled[numeric])

model = RandomForestClassifier(random_state=12345, n_estimators= best_est_forest, max_depth=best_depth_forest)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print('Метрика F1-меры после уменьшения выборки отрицательного класса равна', f1_score(target_valid, predicted_valid))

probabilities_valid = model_linear.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)
print(confusion_matrix(target_valid, predicted_valid))

Как видим, нам не удалось достичь нужного значения F1-меры. Стоит попробовать привести баланс классов к 1:1 путём небольшого увеличения выборки положительного класса и небольшого уменьшения выборки отрицательного класса

In [ ]:
#Приведение выборок к соотношению 1:1

features_upsampled, target_upsampled = upsample(features_train, target_train, 3)
features_downsampled, target_downsampled = downsample(features_upsampled, target_upsampled, 0.75)

print(features_downsampled.shape)
print(target_downsampled.shape)

In [ ]:
#Приведение выборок к соотношению 1:1 и проверка на модели случайного леса

scaler = StandardScaler()
scaler.fit(features_downsampled[numeric])
features_downsampled[numeric] = scaler.transform(features_downsampled[numeric])

model = RandomForestClassifier(random_state=12345, n_estimators= best_est_forest, max_depth=best_depth_forest)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print('Метрика F1-меры после увеличения выборки положительного класса равна', f1_score(target_valid, predicted_valid))
probabilities_valid = model_linear.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)
print(confusion_matrix(target_valid, predicted_valid))

Нужное значение достигнуто, однако имеет смысл ещё раз перебором найти наилучшие гиперпараметры для модели на полученных выборках для увеличения её качества

In [ ]:
best_result_forest = 0
best_model_forest = None
best_depth_forest = 0
best_est_forest = 0
for est in range(10, 121, 10):
    for depth in range(1, 16):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced')
        model_forest.fit(features_downsampled, target_downsampled)
        predicted_valid = model_forest.predict(features_valid)
        result = f1_score(predicted_valid, target_valid)
        if result > best_result_forest:
            best_model_forest = model_forest
            best_result_forest = result
            best_depth_forest = depth
            best_est_forest = est
            
print('F1 наилучшей модели случайного леса на валидационной выборке:', best_result_forest, 
      'Количество деревьев:', best_est_forest, 'Глубина дерева:', best_depth_forest)

probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)

## Тестирование модели

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_est_forest, 
                               max_depth=best_depth_forest, class_weight='balanced')
model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_test)
result = f1_score(predictions, target_test)
print('F1 наилучшей модели случайного леса на тестовой выборке:', result)
if result > 0.59:
    print('Нужное значение F1 меры достигнуто')
else:
    print('Нужное значение F1 меры не достигнуто')

probabilities_valid = best_model_forest.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC равна:', auc_roc)

<b>Вывод</b>

Необходимое значение F1-меры достигнуто. Стоит также отметить, что судя по метрике AUC-ROC полученная модель довольно сильно отличается от случайной модели в лучшую сторону, что говорит о её адекватности.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*